In [1]:
%load_ext autoreload

%autoreload 2
import numpy as np
from scipy.stats import chi2
import numba
from robupy.auxiliary import get_worst_case_probs
from ruspy.estimation.estimation_cost_parameters import (
    lin_cost,
    cost_func,
    create_transition_matrix,
    calc_fixp
)
from scripts.worst_case_probs import calc_fixp_worst, create_worst_trans_mat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
num_states = 50
beta = 0.9999
params = np.array([10, 2.7])
costs = cost_func(num_states, lin_cost, params)
p_ml = np.array([0.39189189, 0.59529357, 0.01281454])
omega = 0.95
rho = chi2.ppf(omega, len(p_ml) - 1) / (2 * 4292)
# rho = 0
threshold = 1e-8

In [9]:
worst_ev, worst_trans_mat = calc_fixp_worst(num_states, p_ml, costs, beta, rho, threshold=threshold)

In [3]:
trans_mat = create_transition_matrix(num_states, p_ml)
ev = calc_fixp(num_states, trans_mat, costs, beta, threshold=threshold)

In [17]:
rho_ = 1
q = np.array([0.5])
v = np.array([-5, -10])
get_worst_case_probs(v, q, rho_)

[0.5]


AssertionError: 

In [26]:
@numba.jit(nopython=True)
def test_func(x):
    if len(x) == 1:
        print(x.shape)

In [27]:
test_func(q)

(1,)


In [8]:
trans_mat_test = create_worst_trans_mat(trans_mat, ev, rho)

[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.37416435 0.61168703 0.01414862]
[0.39189189 0.59529357 0.01281454]
[0.37416435 0.61168703 0.01414862]
[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.37416391 0.6116879  0.01414819]
[0.39189189 0.59529357 0.01281454]
[0.37416391 0.6116879  0.01414819]
[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.37416354 0.61168871 0.01414774]
[0.39189189 0.59529357 0.01281454]
[0.37416354 0.61168871 0.01414774]
[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.37416303 0.61168968 0.01414729]
[0.39189189 0.59529357 0.01281454]
[0.37416303 0.61168968 0.01414729]
[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.3741625  0.61169069 0.01414682]
[0.39189189 0.59529357 0.01281454]
[0.3741625  0.61169069 0.01414682]
[0.39189189 0.59529357 0.01281454]
[0.39189189 0.59529357 0.01281454]
0
[0.37416208 0.6116916  0.01414631]
[0.39189

/home/maxbl/Uni/Master_Thesis/analysis/submodules/robupy/robupy/auxiliary.py:26: RuntimeWarning: overflow encountered in exp
  arg_ = np.maximum(np.sum(q * np.exp(v / lambda_)), EPS_FLOAT)
/home/maxbl/Uni/Master_Thesis/analysis/submodules/robupy/robupy/auxiliary.py:40: RuntimeWarning: overflow encountered in exp
  p = q * np.minimum(np.exp(v / lambda_), MAX_FLOAT)


AssertionError: 

In [6]:
np.sum([1])

1

In [11]:
def Kullback_Leibler(p, p_ml):
    """
    This function calculate the Kullback Leibler Divergence of p and p_ml.
    :param p:
    :param p_ml:
    :return:
    """
    return np.sum(np.multiply(p, np.log(np.divide(p, p_ml))))

In [18]:
Kullback_Leibler(worst_trans_mat[25, 25:28], p_ml)

0.0003713025874300133

In [19]:
rho

0.0006979804924403517